In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
import plotly.express as px

In [45]:
# Infection day distribution based on symptoms of index

viral_loads = pd.read_csv('data/viral_load_mc.csv')

viral_load_s = viral_loads.loc[lambda x: x['14'] != 0]
viral_load_s['symptom_day'] = viral_load_s['14'].map(lambda y: np.round(y))
total_s = viral_load_s.shape[0]

count = []
for t in range(14):
   viral_load_s_i = viral_load_s.loc[lambda x: x[str(t)] > 6]
   viral_load_s_i['diff'] = viral_load_s_i['symptom_day'] - t
   count.append(viral_load_s_i['diff'].value_counts())

   pdf = {i: 0 for i in range(-10, 10)}

for s in count:
    for i in s.index:
        pdf[i] = pdf[i] + s[i]

pdf2 = {i: pdf[i] / total_s for i in pdf.keys() if (i >= 0 and pdf[i] > 0)}
pdf3 = {d: pdf2[d] / sum(pdf2.values()) for d in pdf2}
pdf4 = {d: np.prod([1 - pdf3[i] for i in pdf3 if i > d]) for d in pdf3}
pdf5 = {d: pdf3[d] * pdf4[d] for d in pdf4}
{d: pdf5[d] / sum(pdf5.values()) for d in pdf5}

{0: 0.1669353401405535,
 1: 0.22514759942847082,
 2: 0.26480058806134404,
 3: 0.21587741781643335,
 4: 0.10776722889318635,
 5: 0.01918233381609943,
 6: 0.0002894918439125493}

In [53]:
{d: antigen_detectable[d] / 1e6 for d in antigen_detectable}

{2: 0.071797,
 3: 0.818894,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 0.999998,
 8: 0.997624,
 9: 0.977091,
 10: 0.927458,
 11: 0.845951,
 12: 0.739245,
 13: 0.616213}

In [102]:
# Infection day distribution based on antigen positive of index

# viral_loads = pd.read_csv('data/viral_load_mc.csv').iloc[:int(1e6)].rename(columns={'Unnamed: 0': 'id'})

prob_P = {}
for d in range(1, 13):
    for t in range(d, 13):
        viral_load_p = viral_loads.loc[lambda x: x[str(d - 1)] < 5].loc[lambda x: x[str(d)] >= 5].loc[lambda x: x[str(t)] >= 5].loc[lambda x: x[str(t + 1)] < 5]
        prob_P[(d, t)] = viral_load_p.shape[0] / 1e6
for d in range(1, 12):
    viral_load_p = viral_loads.loc[lambda x: x[str(d - 1)] < 5].loc[lambda x: x[str(d)] >= 5].loc[lambda x: x['13'] >= 5]
    prob_P[(d, 13)] = viral_load_p.shape[0] / 1e6

prob_t = {t: sum([prob_P[d] / (d[1] - d[0] + 1) for d in prob_P if d[0] <= t and t <= d[1]]) for t in range(2, 14)}

count_days = {}
for d in range(2, 14):
    viral_load_d = viral_loads.loc[lambda x: x[str(d)] >= 5]
    viral_load_d = viral_load_d[['id'] + [str(t) for t in range(0, d + 1)]]
    viral_load_d = pd.melt(viral_load_d, id_vars=['id'], var_name='day', value_name='viral_load')
    viral_load_d = viral_load_d.loc[lambda x: x['viral_load'] >= 6]
    count_days[d] = viral_load_d['day'].value_counts()

pdf = {d: count_days[d] / count_days[d].sum() for d in range(2, 14)}
pdf2 = {d: {pdf2[d][i] = pdf[d][str(i)] for i in range(2, d + 1)} for d in pdf}
pdf3 = {d: {t: pdf2[d][t] * np.prod([1 - pdf2[d][i] for i in pdf2[d] if i < t]) for t in pdf2[d]} for d in pdf2}
pdf4 = {d: {d - t: pdf3[d][t] / sum(pdf3[d].values()) for t in pdf3[d]} for d in pdf3}
pdf5 = {i: sum([prob_t[t]* pdf4[t][i]  for t in prob_t if t - 1 > i]) for i in range(14)}
pdf5

{0: 0.22814274601440315,
 1: 0.18036607548052103,
 2: 0.14579768147486516,
 3: 0.11949771919192992,
 4: 0.09742601719039967,
 5: 0.07776776000225924,
 6: 0.05958155718786974,
 7: 0.04287953153982928,
 8: 0.028116636543746347,
 9: 0.015361660350072108,
 10: 0.004938378035518153,
 11: 0.00012423698858623843,
 12: 0,
 13: 0}

In [30]:
pdf

{2: 2    1.0
 Name: day, dtype: float64,
 3: 3    0.959457
 2    0.040543
 Name: day, dtype: float64,
 4: 4    0.611888
 3    0.372377
 2    0.015735
 Name: day, dtype: float64,
 5: 5    0.387879
 4    0.374549
 3    0.227940
 2    0.009632
 Name: day, dtype: float64,
 6: 5    0.279476
 6    0.279476
 4    0.269872
 3    0.164236
 2    0.006940
 Name: day, dtype: float64,
 7: 5    0.218675
 6    0.218675
 7    0.217554
 4    0.211160
 3    0.128506
 2    0.005430
 Name: day, dtype: float64,
 8: 5    0.180951
 6    0.180951
 7    0.180395
 4    0.174718
 8    0.172504
 3    0.106160
 2    0.004321
 Name: day, dtype: float64,
 9: 5    0.155932
 6    0.155932
 7    0.155785
 8    0.151722
 4    0.150447
 9    0.136609
 3    0.090132
 2    0.003440
 Name: day, dtype: float64,
 10: 5     0.138502
 6     0.138502
 7     0.138462
 8     0.137084
 4     0.133370
 9     0.127790
 10    0.105952
 3     0.077543
 2     0.002795
 Name: day, dtype: float64,
 11: 5     0.125789
 6     0.125789
 7   

In [18]:
viral_loads

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,-3.4,-0.1,3.2,6.5,8.6,8.2,7.8,7.5,7.1,6.8,6.4,6.1,5.7,5.3,0.000000
1,1,-5.4,-0.2,5.0,9.4,8.7,7.9,7.2,6.4,5.7,4.9,4.1,3.4,2.6,1.9,0.000000
2,2,-2.1,0.4,3.0,5.5,8.0,8.8,8.5,8.1,7.8,7.5,7.2,6.9,6.6,6.2,5.895681
3,3,-4.1,-0.1,3.8,7.5,7.2,7.0,6.8,6.6,6.4,6.2,6.0,5.8,5.6,5.4,0.000000
4,4,-4.0,0.1,4.1,8.2,8.5,8.1,7.8,7.4,7.0,6.6,6.2,5.8,5.4,5.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,-7.4,-1.7,4.0,9.6,9.3,8.9,8.4,7.9,7.5,7.0,6.6,6.1,5.7,5.2,4.869682
999996,999996,-3.8,-0.4,3.1,6.5,10.0,9.4,8.8,8.2,7.6,7.0,6.4,5.9,5.3,4.7,6.962723
999997,999997,-9.5,-4.2,1.1,6.5,9.1,8.6,8.2,7.7,7.2,6.8,6.3,5.8,5.4,4.9,5.233132
999998,999998,-4.1,-0.6,2.8,6.3,8.2,7.9,7.6,7.3,7.0,6.7,6.4,6.1,5.7,5.4,0.000000


In [2]:
# Pareto Points

res = pd.read_csv('data/results_23_05_2.csv')

def pareto(df):
    aux = df.sort_values('expected_infecting_days', ascending=True)

    pareto_points = []    
    while aux.shape[0] > 0:
        pareto_points.append(aux.iloc[0:1])
        current_high = aux['expected_non_infecting_days'].iloc[0]
        aux = aux.loc[lambda x: x['expected_non_infecting_days'] > current_high]
        
    return pd.concat(pareto_points)

pareto_points = res.groupby(['num_antigen', 'num_pcr'], as_index=False).apply(pareto)

pareto_points.to_csv('data/pareto_points_23_05_2.csv')


In [97]:
# Table of best policies

res = pd.read_csv('data/results_19_05_1.csv').rename(columns={'variance_infencting_days': 'variance_infecting_days'})

def best_k(df, k=3):
    best_k_df = df.sort_values('expected_infecting_days', ascending=True).iloc[:k]
    return best_k_df

def best_p(df, p=.05):
    best_value = df.sort_values('expected_infecting_days', ascending=True)['expected_infecting_days'].iloc[0]
    best_p_df = df.loc[df['expected_infecting_days'] < (1 + p) * best_value]
    return best_p_df

def best_p_and_k(df, p=0.2, k=1):
    best_value = df.sort_values('expected_infecting_days', ascending=True)['expected_infecting_days'].iloc[0]
    best_p_df = df.loc[df['expected_infecting_days'] < (1 + p) * best_value]
    best_p_k_df = (
        best_p_df
        .sort_values('expected_infecting_days', ascending=True)
        .iloc[:k]
        .sort_values('variance_infecting_days', ascending=True)
        .assign(ranking = lambda x: list(range(1, x.shape[0] + 1)))
    )

    return best_p_k_df

def best_sd(df):
    best_value = df.sort_values('expected_infecting_days', ascending=True)['expected_infecting_days'].iloc[0]
    best_value_sd = df.sort_values('expected_infecting_days', ascending=True)['variance_infecting_days'].iloc[0]
    best_sd_df = df.loc[df['expected_infecting_days'] < best_value_sd + best_value]
    return best_sd_df


# (res
#     .groupby([ 'num_pcr', 'num_antigen'], as_index=False)
#     .apply(best_p_and_k)
#     # .set_index([ 'num_pcr', 'num_antigen', 'ranking'])
#     .assign(policy = lambda x: 'Antigen: ' + x['antigen_days'] + ', PCR: ' + x['pcr_days'])
#     .pivot(index=['num_pcr', 'num_antigen'],
#            columns='ranking',
#            values=['policy'])
#     .fillna('[]')
#     .applymap(lambda x: x.replace('[]', ''))
#     .to_csv('data/best_policies_table_20_05_2.csv')
# )

(res
    .groupby([ 'num_pcr', 'num_antigen'], as_index=False)
    .apply(best_p_and_k)
    # .set_index([ 'num_pcr', 'num_antigen', 'ranking'])
    .assign(policy = lambda x: 'Antigen: ' + x['antigen_days'] + ', PCR: ' + x['pcr_days'])
    .pivot(index=['num_pcr', 'num_antigen'],
           columns='ranking',
           values=['expected_infecting_days', 'expected_non_infecting_days'])
    .fillna(0)
    # .applymap(lambda x: x.replace('[]', ''))
    .to_csv('data/best_performances_table_20_05_1.csv')
)

# res_aux = res.groupby([ 'num_pcr', 'num_antigen',], as_index=False).apply(best_p_and_k)#.assign(antigen_days = lambda x: 'Antigen: ' + x['antigen_days'], pcr_days = lambda x: 'PCR: ' + x['pcr_days'])

# res1 = res_aux.loc[lambda x: x['ranking'] == 1][['num_pcr', 'num_antigen', 'antigen_days', 'pcr_days', 'ranking']]
# res2 = res_aux.loc[lambda x: x['ranking'] == 2][['num_pcr', 'num_antigen', 'antigen_days', 'pcr_days', 'ranking']]
# res3 = res_aux.loc[lambda x: x['ranking'] == 3][['num_pcr', 'num_antigen', 'antigen_days', 'pcr_days', 'ranking']]

# res1.merge(res2, on=['num_pcr', 'num_antigen']).merge(res3, on=['num_pcr', 'num_antigen']).sort_values(['num_pcr', 'num_antigen']).set_index(['num_pcr', 'num_antigen'])

# aux = res.groupby([ 'num_pcr', 'num_antigen',], as_index=False).apply(best_p_and_k).assign(antigen_days = lambda x: 'Antigen: ' + x['antigen_days'], pcr_days = lambda x: 'PCR: ' + x['pcr_days']).groupby([ 'num_pcr', 'num_antigen', 'ranking']).agg({'antigen_days': 'first', 'pcr_days':'first'}).unstack()
# aux.columns = aux.columns.swaplevel(0, 1)
# aux.sort_index(level=0, axis=1).fillna('[]').applymap(lambda x: x.replace('[]', '')).to_csv('data/best_policies_table_20_05_1.csv')

In [30]:
pareto_points.assign(combo = lambda x: str(y['num_antigen']) + ',' + str(y['num_pcr']))

NameError: name 'y' is not defined

In [37]:
pdf2

{0: 1.0,
 1: 0.9989026309458073,
 2: 0.9002216137504923,
 3: 0.6164350165325337,
 4: 0.2849591641515709,
 5: 0.050062788229371935,
 6: 0.0007553786983207074}